## Importations des bibliothèques


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from datetime import datetime
import warnings


## Chargement des données


In [23]:
# Charger les données
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
print(train_df.columns)
print(test_df.columns)



Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


## Nettoyage de Données

In [24]:
warnings.filterwarnings('ignore')

# Vérifier la présence de la colonne 'Embarked'
if 'Embarked' in train_df.columns:
    # Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente (si présente)
    train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans train_df")

if 'Embarked' in test_df.columns:
    test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans test_df")

# Remplir les valeurs manquantes pour 'Age' avec la médiane
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# Remplir les valeurs manquantes pour 'Fare' dans le jeu de test avec la médiane
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Supprimer la colonne 'Cabin' car elle contient trop de valeurs manquantes
train_df.drop('Cabin', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

# Créer une nouvelle feature 'FamilySize' à partir de 'SibSp' et 'Parch'
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Créer une nouvelle feature 'IsAlone' à partir de 'FamilySize'
train_df['IsAlone'] = 1
train_df.loc[train_df['FamilySize'] > 1, 'IsAlone'] = 0
test_df['IsAlone'] = 1
test_df.loc[test_df['FamilySize'] > 1, 'IsAlone'] = 0

# Supprimer les colonnes inutiles
train_df = train_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)

# Convertir la feature 'Sex' en numérique
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

# Convertir la feature 'Embarked' en numérique en utilisant le one-hot encoding
train_df = pd.get_dummies(train_df, columns=['Embarked'])
test_df = pd.get_dummies(test_df, columns=['Embarked'])


##  Entraîner le modèle avec MLflow

In [25]:
 # Séparer les données d'entraînement
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Démarrer une nouvelle expérience MLflow
mlflow.set_experiment('Titanic')

with mlflow.start_run() as run:
    # Définir un nom pour le run avec la date et l'heure actuelles
    run_name = datetime.now().strftime("Run_%Y-%m-%d_%H-%M-%S")
    client = MlflowClient()
    client.set_tag(run.info.run_id, "mlflow.runName", run_name)
    # Entraîner le modèle
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Évaluer le modèle
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print(f'Validation Accuracy: {accuracy}')
    
    # Enregistrer les paramètres et les métriques dans MLflow
    mlflow.log_param('n_estimators', 100)
    mlflow.log_param('random_state', 42)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.sklearn.log_model(model, 'model')
    
    if accuracy > 0.84:
        
        model_uri = f'runs:/{mlflow.active_run().info.run_id}/model'
        print(f'Model URI: {model_uri}')
        
        # Creation du modèle
        model_name = "TitanicModel2"
        
        try:
            client.create_registered_model(model_name)
        except mlflow.exceptions.MlflowException as e:
            print(f"Registered model '{model_name}' already exists or another error occurred: {e}")
        
        model_version = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run.info.run_id
        )

        print(f'Model version {model_version.version} with run_id {run.info.run_id} is registered to production.')
    else:
        print('Model accuracy is below the threshold. Model not registered.')




Validation Accuracy: 0.8435754189944135
Model URI: runs:/80184fa45d4443ed884290d691e7a43f/model
Model version 1 with run_id 80184fa45d4443ed884290d691e7a43f is registered to production.


## Prédictions sur le jeu de test et créer le fichier de soumission

In [26]:
# Faire des prédictions sur le jeu de test
predictions = model.predict(test_df)

# Créer un fichier de soumission
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
submission.to_csv('submission.csv', index=False)